In [1]:
import shutil
from os import path, getcwd, chdir, makedirs, listdir
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

class_names = df.species.unique()
num_classes = len(class_names)

# setup folders for each class

for i in range(len(df)):
    Class = df.iloc[i,1]
    Identity = df.iloc[i,2]
    File= df.iloc[i,0]
    filepath = path.join(f'test/{Class}',Identity)
    file=path.join(filepath,File)
    makedirs(filepath, exist_ok=True)
    if not path.exists(file):
        shutil.copy(f'train_images/all/{File}',filepath)

In [24]:
img_height = 128
img_width = 128


model = models.Sequential([
    
    layers.Conv2D(filters=32, kernel_size=(5,5), activation= 'relu', input_shape=(img_height,img_width,3)),
    layers.MaxPooling2D(2,2,),

        

    layers.Flatten(),

    layers.Dense(256, activation='relu'),
    layers.Dense(num_classes, activation='softmax')

])
#model.summary()
opt = optimizers.Adam(.001)
model.compile(optimizer = opt,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


checkpoint_filepath = './tmp/checkpoint.ckpt'
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    verbose = 1,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


# The model weights (that are considered the best) are loaded into the model.
#model.load_weights(checkpoint_filepath)


In [23]:
batch_size = 100
img_height
img_width

train_datagen = ImageDataGenerator(rescale=1/255,
#    rotation_range=20,
#    zoom_range=.2,
#    shear_range=20,
#   horizontal_flip=True,
#   fill_mode='nearest',                  
    validation_split=0.2)


train_generator = train_datagen.flow_from_directory(
    'images',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    'images', # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') 

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 2,
    callbacks=[model_checkpoint_callback])

history.epoch, history.history['accuracy'][-1]

Found 40839 images belonging to 30 classes.
Found 10194 images belonging to 30 classes.
Epoch 1/2
408/408 [==============================] - ETA: 0s - loss: 1.8193 - accuracy: 0.5432
Epoch 1: val_accuracy improved from -inf to 0.62040, saving model to ./tmp\checkpoint.ckpt
408/408 [==============================] - 1061s 3s/step - loss: 1.8193 - accuracy: 0.5432 - val_loss: 1.3066 - val_accuracy: 0.6204
Epoch 2/2
408/408 [==============================] - ETA: 0s - loss: 0.9674 - accuracy: 0.7103
Epoch 2: val_accuracy improved from 0.62040 to 0.66178, saving model to ./tmp\checkpoint.ckpt
408/408 [==============================] - 1051s 3s/step - loss: 0.9674 - accuracy: 0.7103 - val_loss: 1.1608 - val_accuracy: 0.6618


([0, 1], 0.7103267312049866)